## Loading packages and dataset

In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential
import keras.utils as ku
from keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [30]:
with open('metamorphosis_clean.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [31]:
print(text[:600])

One morning, when Gregor Samsa woke from troubled dreams, he found
himself transformed in his bed into a horrible vermin. He lay on his
armour-like back, and if he lifted his head a little he could see his
brown belly, slightly domed and divided by arches into stiff sections.
The bedding was hardly able to cover it and seemed ready to slide off
any moment. His many legs, pitifully thin compared with the size of the
rest of him, waved about helplessly as he looked.

“What’s happened to me?” he thought. It wasn’t a dream. His room, a
proper human room although a little too small, lay peacefully 


In [6]:
len(text)

118412

## Data preprocessing

In [7]:
# Tokenizing the text (word to vectors and vectors to word)

chars = sorted(list(set(text)))
char_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_char = {i:ch for i, ch in enumerate(chars)}
no_of_vocab = len(chars)

In [8]:
print(chars)
no_of_vocab

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', '—', '’', '“', '”']


64

In [9]:
# Set the maximum sequence length (max_len) to be the length of the longest sequence

max_len = len(max(text.split('\n'), key=len))
# max_len = 40

# Creating training data vectors and label vectors
X = []
y = []

for i in range(0, len(text)-max_len, 1):
    X.append([char_to_int[ch] for ch in text[i:i+max_len]])
    y.append(char_to_int[text[i+max_len]])
no_of_patterns = len(X)

no_of_patterns, max_len

(118341, 71)

In [10]:
# Padding the training sequences to the same length
X = pad_sequences(X, maxlen=max_len, padding='post')

# Converting labels to categorical format
y = ku.to_categorical(y)

In [11]:
X[0], y[0]

(array([24, 46, 37,  1, 45, 47, 50, 46, 41, 46, 39,  5,  1, 55, 40, 37, 46,
         1, 17, 50, 37, 39, 47, 50,  1, 27, 33, 45, 51, 33,  1, 55, 47, 43,
        37,  1, 38, 50, 47, 45,  1, 52, 50, 47, 53, 34, 44, 37, 36,  1, 36,
        50, 37, 33, 45, 51,  5,  1, 40, 37,  1, 38, 47, 53, 46, 36,  0, 40,
        41, 45, 51], dtype=int32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [12]:
# Checking the shape of the training data and label vectors
X.shape[0], X.shape[1], y.shape[0], y.shape[1]

(118341, 71, 118341, 64)

## Model training

In [23]:
# Defining the model architecture

model = Sequential()
model.add(Embedding(input_dim=no_of_vocab, output_dim=64))
model.add(LSTM(units=256, return_sequences=True))              # , activation='tanh', recurrent_activation='sigmoid', return_sequences=True
model.add(LSTM(units=128))
model.add(Dense(units=no_of_vocab, activation='softmax'))
optim = Adam(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          4096      
                                                                 
 lstm (LSTM)                 (None, None, 256)         328704    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
Total params: 538176 (2.05 MB)
Trainable params: 538176 (2.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Training the model
model.fit(X, y, epochs=100, batch_size=64, verbose=1)

Epoch 1/100


2024-01-23 01:29:06.645775: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-23 01:29:07.015305: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f495c00db00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 01:29:07.015382: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro M1200, Compute Capability 5.0
2024-01-23 01:29:07.048264: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705998547.210125    4616 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1850/1850 [==============================] - 92s 45ms/step - loss: 2.3177 - accuracy: 0.3460
Epoch 2/100
1850/1850 [==============================] - 80s 43ms/step - loss: 1.8304 - accuracy: 0.4624
Epoch 3/100
1850/1850 [==============================] - 79s 42ms/step - loss: 1.8975 - accuracy: 0.4461
Epoch 4/100
1850/1850 [==============================] - 79s 42ms/step - loss: 1.6955 - accuracy: 0.4955
Epoch 5/100
1850/1850 [==============================] - 78s 42ms/step - loss: 1.5612 - accuracy: 0.5331
Epoch 6/100
1850/1850 [==============================] - 79s 42ms/step - loss: 1.4804 - accuracy: 0.5545
Epoch 7/100
1850/1850 [==============================] - 79s 42ms/step - loss: 1.4549 - accuracy: 0.5611
Epoch 8/100
1850/1850 [==============================] - 79s 43ms/step - loss: 1.3922 - accuracy: 0.5760
Epoch 9/100
1850/1850 [==============================] - 79s 42ms/step - loss: 1.3422 - accuracy: 0.5908
Epoch 10/100
1850/1850 [==============================] - 78s 42ms/

In [ ]:
# Saving the model
model.save('model_small.h5')

In [15]:
# Loading the model for inference

model.load_weights('model_small.h5')
model.compile(loss='categorical_crossentropy', optimizer='Adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          4096      
                                                                 
 lstm (LSTM)                 (None, None, 256)         328704    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
Total params: 538176 (2.05 MB)
Trainable params: 538176 (2.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Alternative function to generate text

# def generate_text(seed, num_chars):
#     # Initialize the generated text
#     generated_text = seed
#     # Encode the seed as integers
#     encoded_seed = [char_to_int[char] for char in seed]
#     # Pad the seed
#     padded_seed = pad_sequences([encoded_seed], maxlen=max_len, padding='post')
#     # Generate characters
#     for i in range(num_chars):
#         # Get the next character probabilities
#         predictions = model.predict(padded_seed, verbose=0)[0]
#         # Get the index of the character with the highest probability
#         index = np.argmax(predictions)
#         # Add the character to the generated text
#         generated_text += int_to_char[index]
#         # Update the padded seed with the latest character
#         padded_seed = np.append(padded_seed[0][1:], index)
#         padded_seed = pad_sequences([padded_seed], maxlen=max_len, padding='post')
#
#     return generated_text

## Model inference

In [17]:
# Function to generate text

def generate_text(num_chars, lstm_model, random_text=None):
    # pick a random seed
    random_text = np.random.randint(0, len(X)-1)
    pattern = X[random_text]
    seed = ''.join([int_to_char[value] for value in pattern])
    print("Seed:")
    print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    print("Generated text:\n")
    # Initialize the generated text
    generated_text = ''
    # Encode the seed as integers
    encoded_seed = [char_to_int[char] for char in seed]
    # Pad the seed
    padded_seed = pad_sequences([encoded_seed], maxlen=max_len, padding='post')
    # Generate characters
    for _ in range(num_chars):
        # Get the next character probabilities
        predictions = lstm_model.predict(padded_seed, verbose=0)[0]
        # Get the index of the character with the highest probability
        index = np.argmax(predictions)
        # Add the character to the generated text
        generated_text += int_to_char[index]
        # Update the padded seed with the latest character
        padded_seed = np.append(padded_seed[0][1:], index)
        padded_seed = pad_sequences([padded_seed], maxlen=max_len, padding='post')
        
    return generated_text

In [16]:
# Generate some text with specified length using random seed text from the data

# generated_text = generate_text(seed="Rejoice", num_chars=200)
generated_text = generate_text(num_chars=600, lstm_model=model)
print(generated_text)

Seed:
" im. And so he ran up to his father,
stopped when his father stopped, sc "
Generated text:

urried forwards again when he stretched and she had to
deat set aside for exes with his father’s behaviour that she was no
pleasure for Gregor’s room as a sign of his pleasure, that was something that they had hoped there was already in tears that they had been standing there.

Gregor then there was no immediate notice price without delaying as
they landed looked as if he was clearly as he had sending avoiring
anything a lack of his body to a state. But his father as if they had
become much lighter at his sister was a sill-or whether it was a sill
and most of what they were even a long time ag


## Larger model architecture (training and inference)

In [18]:
# Defining the larger LSTM model architecture

model_large = Sequential()
model_large.add(Embedding(input_dim=no_of_vocab, output_dim=80))
model_large.add(LSTM(1024, return_sequences=True))
model_large.add(Dropout(0.2))
model_large.add(LSTM(128, return_sequences=True))
model_large.add(Dropout(0.2))
model_large.add(LSTM(128))
model_large.add(Dropout(0.2))
model_large.add(Dense(units=no_of_vocab, activation='softmax'))
model_large.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

filepath="best-model-weights-{epoch:02d}-{loss:.4f}-{accuracy:.2f}.hdf5"

# Defining the checkpoints and callbacks
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=0.001)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint, reduce_lr]

In [19]:
model_large.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 80)          5120      
                                                                 
 lstm_2 (LSTM)               (None, None, 1024)        4526080   
                                                                 
 dropout (Dropout)           (None, None, 1024)        0         
                                                                 
 lstm_3 (LSTM)               (None, None, 128)         590336    
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_4 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [ ]:
# Training the larger model
model_large.fit(X, y, epochs=100, batch_size=64, verbose=1, callbacks=desired_callbacks)

Epoch 1/100
1850/1850 [==============================] - ETA: 0s - loss: 2.7980 - accuracy: 0.2222
Epoch 1: loss improved from inf to 2.79799, saving model to best-model-weights-01-2.7980-0.22.hdf5
1850/1850 [==============================] - 117s 59ms/step - loss: 2.7980 - accuracy: 0.2222 - lr: 0.0010
Epoch 2/100
1850/1850 [==============================] - ETA: 0s - loss: 2.2182 - accuracy: 0.3652
Epoch 2: loss improved from 2.79799 to 2.21816, saving model to best-model-weights-02-2.2182-0.37.hdf5
1850/1850 [==============================] - 108s 58ms/step - loss: 2.2182 - accuracy: 0.3652 - lr: 0.0010
Epoch 3/100
1850/1850 [==============================] - ETA: 0s - loss: 1.9708 - accuracy: 0.4273
Epoch 3: loss improved from 2.21816 to 1.97085, saving model to best-model-weights-03-1.9708-0.43.hdf5
1850/1850 [==============================] - 108s 58ms/step - loss: 1.9708 - accuracy: 0.4273 - lr: 0.0010
Epoch 4/100
1850/1850 [==============================] - ETA: 0s - loss: 1.80

In [ ]:
model_large.save('model_large.h5')

In [27]:
# Loading the model weights

# model_large.load_weights('model_large.h5')
filename = "best-model-weights-100-0.7908-0.75.hdf5" 
model_large.load_weights(filename)
model_large.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
# Generating some text from random seed text

generated_text = generate_text(num_chars=600, lstm_model=model_large)
print(generated_text)

Seed:
" leman, “would the young lady not like to
come in and play for us here i "
Generated text:

n the floor. He would not be tomened to be someone from his father and sister to think about the chief clerk them with the strength and then had been slightly unwell then, and without decisively or at the contraight away, as he was still have to be some of them and still because he was so much more some of that he was still have to be the same time. They had to be seen in a chair and some super family then had been suddenly come in that he was something while his sister was something that he was something that he would have been struck in the flat and slightest happening, he was something with
